In [ ]:
#Instalar Google Cloud
#!pip install gcloud
#!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
#!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
#!apt -qq update
#!apt -qq install gcsfuse

# Instalar PIL para resizing imagenes
#!pip install PIL

#Conexion Cloud
from google.colab import auth
auth.authenticate_user()

#https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'angelic-cat-338018'
!gcloud config set project {project_id}

# Cargamos Dataset usado por Nacho para ML - Conexion Drive 
# Importar drive y pandas
from google.colab import drive
drive.mount('/content/drive')

# Instalar zipfile
# !pip install zipfile36

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Mounted at /content/drive


In [ ]:
# Librerias
import pandas as pd
import os
import numpy as np
import zipfile
from PIL import Image
import math
import timeit

In [ ]:
# Definir el directorio de drive en el que está el dataset
# !!!!!!!!!!!!!!!!!!!!!!!!!!!
# Para los que no somos Alvaro (owner de la carpeta "TFM Big DAta & BA") en Drive tenemos que hacer click derecho en ella > Añadir acceso a Drive > "Mi Unidad", para que se indexe y funcione la ruta)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!

#directorio_dataset = '/content/drive/MyDrive/TFM Big Data & BA/Propuestas/Proyecto Final/0 - Dataset/train_t.csv'
directorio_dataset = '/content/drive/MyDrive/TFM Big Data & BA/Propuestas/Proyecto Final/0 - Dataset/train_data.csv'

df = pd.read_csv(directorio_dataset)

In [ ]:
#Prueba para buscar cosas en el df
'8513a91e55670c709069b5f85e12a59095b802877715903abef16b7a6f306e58' in df.image.values
'60d310a42e87cdf799afcd89dc1b11ae3fdc3d0233747ec7ef78d82c87002e83' in df.image.values
'b98b291bd04c3d92165ca515e00468fd9756af9a8f1df42505deed1dcfb5d7ae' in df.image.values

False

In [ ]:
#Eliminar registro de train_t con foto dañada 8513a91e55670c709069b5f85e12a59095b802877715903abef16b7a6f306e58.jpg
df = df.drop(df[df['image']=='8513a91e55670c709069b5f85e12a59095b802877715903abef16b7a6f306e58'].index)
df = df.drop(df[df['image']=='60d310a42e87cdf799afcd89dc1b11ae3fdc3d0233747ec7ef78d82c87002e83'].index)
df = df.drop(df[df['image']=='b98b291bd04c3d92165ca515e00468fd9756af9a8f1df42505deed1dcfb5d7ae'].index)


In [ ]:
#Prueba para buscar cosas en el df
'8513a91e55670c709069b5f85e12a59095b802877715903abef16b7a6f306e58' in df.image.values
'60d310a42e87cdf799afcd89dc1b11ae3fdc3d0233747ec7ef78d82c87002e83' in df.image.values
'b98b291bd04c3d92165ca515e00468fd9756af9a8f1df42505deed1dcfb5d7ae' in df.image.values

False

In [ ]:
df

,Unnamed: 0,id,price,user_type,deal_probability,category,subcategory,city,region,image,description,title,population,wage,federal_district,wage_fd,sold,parent_category
0,0,71fdcaaf5cf2,8.400,Company,0.37064,Tech accessories,For phones,Troitsk,Chelyabinsk region,9ea984545052ffb02c8356c9f5460f58c03fef18507248...,Продам СРОЧНО/\nГарнитура - Динамики для телеф...,Nokia MD - 8 (Original) Black Edition,3442810,473.496,Ural,723,1,Electronics
1,1,e8fd28323e41,1.560,Company,0.16155,Tech accessories,For phones,Yekaterinburg,Sverdlovsk region,dc469d035945552a6fa5d03d5bba2a331a0a4213d72c14...,Зарядное устройство подойдет для любого соврем...,Автомобильное зарядное устройство USB 1А 2А,4290067,512.424,Ural,723,1,Electronics
2,2,1e6ee485816e,3.000,Private,0.21080,Tech accessories,For phones,Taganrog,Rostov region,ef1760e9c2ae46ef0d652a9e5b74874f44bb0f351b0f3d...,"Отличный крепкий чехол для защиты от ударов, о...",Чехол для meizu m5 note,4181486,416.484,Southern,481,1,Electronics
3,3,d7fbb29cf0d3,12.000,Private,0.33973,Tech accessories,For phones,Rostov-on-Don,Rostov region,9961c32699847a15c6894465bffcc547bea6a71fdc673d...,Продам дисплей на смартфон IPhone 5/\nТорг/\nП...,Дисплей на смартфон iPhone 5,4181486,416.484,Southern,481,1,Electronics
4,4,bb231d205c71,3.480,Company,0.26355,Tech accessories,For phones,Ufa,Bashkortostan,46b9cebbe689c1dce3d15b2168f62da7b97b8ea94c3d33...,Аккумулятор для Samsung Galaxy Star Plus Duos ...,Аккумулятор Samsung Galaxy Star Plus Duos GT-S...,4013786,464.628,Volga,479,1,Electronics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622757,622757,88831c445e26,78.000,Private,0.82668,Musical instruments,Accessories,Ust-Ilimsk,Irkutsk region,445e7382b73d128e61d11a9c93ee99ff6747587335b5ca...,Использовался минимально и только в домашних у...,Комбоусилитель Roland Cube 20X,2375021,580.152,Siberian,583,1,Hobbies
622758,622758,d2b690dd70f0,6.000,Private,0.13407,Musical instruments,Accessories,Samara,Samara Region,21f76d802fd3f3ec7b6b0038412af1fb979ed487537cea...,В хорошем сотояние,Чехол на гитару,3154164,452.364,Volga,479,1,Hobbies
622759,622759,e5d0952d9645,7.200,Private,0.00000,Musical instruments,Accessories,Kemerovo,Kemerovo Region,f7e82c979bfaf03459e766ea53ab77ede4018c8fcc7c34...,Продам ремень для гитары. Разные расцветки. От...,Ремень для гитары,2633446,526.788,Siberian,583,0,Hobbies
622760,622760,a2368b4b8ea3,42.000,Private,0.00000,Musical instruments,Accessories,Tula.,Tula region,2f7748a6478c507a40d32a3e5f2b11e74ef797c2fb55c3...,Продаются педали эффектов. Все в отличном сост...,Педали эффектов,1449115,473.736,Central,841,0,Hobbies


# Subida de datos limpios

In [ ]:
# Selección de variables que necesitamos para el join con las imagenes
#df = df[['item_id', 'image', 'cat_category_name']]
df = df[['id', 'image', 'category']]

In [ ]:
# Copiar los zip a local > filtrar imagenes en df Nacho > Subir a cleaned indexed
!mkdir /content/extractions
!mkdir /content/converted

ziprefs = ["/content/train_jpg_0.zip", "/content/train_jpg_1.zip", 
           "/content/train_jpg_2.zip", "/content/train_jpg_3.zip",
           "/content/train_jpg_4.zip"]

In [ ]:
#Original
for zipref in ziprefs:
  
   gsutil_file = zipref.split('/')[2]
   !gsutil cp -r gs://tfmbucket1/images/train/raw/{gsutil_file} {gsutil_file}

   zipactual = zipfile.ZipFile(zipref, 'r')

   output = pd.DataFrame(zipactual.namelist()[1:], columns = ['full_filename'])
   output['image'] = output.apply(lambda x: x['full_filename'].split('.')[0], axis = 1)  #.split('/')[3]
   output['fullpath'] = output.apply(lambda x: '/content/extractions/' + x['image'] + '.jpg', axis = 1)
   output['LowRes'] = output.apply(lambda x: '/content/converted/LowRes_' + x['image'] + '.jpg', axis = 1)

   fulldf = output.merge(df, on='image', how='left')
   cleandf = fulldf.dropna()
  
   #os.remove(zipref)
   zipactual.extractall(path="/content/extractions/", members = cleandf['full_filename'])

   for img in cleandf['full_filename']:
      imgfile = Image.open('/content/extractions/' + img) # image extension *.png,*.jpg
      new_width  = 128
      new_height = 128
      imgfile = imgfile.resize((new_width, new_height), Image.ANTIALIAS)
      imgfile.save('/content/converted/LowRes_' + img)     

   np.savetxt(r'/content/files_to_upload.txt', cleandf['LowRes'], fmt='%s')
   
  # Bulk upload
   !cat /content/files_to_upload.txt | gsutil -m cp -I gs://tfmbucket1/images/train/LowRes_V2

  # One by one upload (mucho mas lento que bulk)
  # for img in cleandf['path']:
  #  zipactual.extract(img, path = '/cleaned/')
  #  !gsutil cp /cleaned/{img} gs://tfmbucket17/images/test/cleaned/{img}
  #  os.remove('/cleaned/' + img)

   !rm -r /content/extractions
   !rm -r /content/converted
   os.remove('/content/files_to_upload.txt')

   !mkdir /content/extractions
   !mkdir /content/converted  

Se han truncado las últimas 5000 líneas del flujo de salida.
Copying file:///content/converted/LowRes_9bc64f8ed563d16988b2016535918940752f57c01c8d8178d3bd28b2bbf92024.jpg [Content-Type=image/jpeg]...
Copying file:///content/converted/LowRes_1a6516f430df00b60552f0ea512320bac8b77ec984b24b4508128277e400c5a3.jpg [Content-Type=image/jpeg]...
Copying file:///content/converted/LowRes_b5914364aec6d125ce2d70d73fa1dc4b24bebfc53508e8e01e54d2a23fec7aca.jpg [Content-Type=image/jpeg]...
Copying file:///content/converted/LowRes_495d3e084dcf5561d4a89c872ff414d5616e288b058444e2caf6e13478dcc5fa.jpg [Content-Type=image/jpeg]...
Copying file:///content/converted/LowRes_ea69318c4da573e698ab0759c6ab9d39eaa2933dae2c98fad3d929c8b2ce1ac5.jpg [Content-Type=image/jpeg]...
Copying file:///content/converted/LowRes_f5207bd6a0b8a5c7f457f406a5dde5ef8fae85fee6c985afb7402e92658b5a06.jpg [Content-Type=image/jpeg]...
Copying file:///content/converted/LowRes_fa0bcb9a69e9515fba08787debc095e99d5599744c26275d27da0ce87aadc1b0

In [ ]:
print (len(fulldf))
print (len(cleandf))
#Prueba para buscar cosas en el fulldf
'8513a91e55670c709069b5f85e12a59095b802877715903abef16b7a6f306e58' in fulldf.image.values
#Prueba para buscar cosas en el cleandf
'8513a91e55670c709069b5f85e12a59095b802877715903abef16b7a6f306e58' in cleandf.image.values

278167
207826


True

In [ ]:
#Descargar Imagenes en directorio raíz
#!gsutil -m cp -r gs://tfmbucket1/images/train/LowRes .

In [ ]:
#Descargar Imagenes en directorio del Drive
!gsutil -m cp -r gs://tfmbucket1/images/train/LowRes '/content/drive/MyDrive/TFM Big Data & BA/Propuestas/Proyecto Final/4 - Deep Learning/.'

Se han truncado las últimas 5000 líneas del flujo de salida.
Copying gs://tfmbucket1/images/train/LowRes/LowRes_5f301e03021121d98c69399ce42a345feb1b5dbda5c2829b2a622c5d5ab05a02.jpg...
Copying gs://tfmbucket1/images/train/LowRes/LowRes_5f303c69814c2fdccb66d203403097e86103b153608a33ab649b7a4b619f70c0.jpg...
Copying gs://tfmbucket1/images/train/LowRes/LowRes_5f304fdacba967117e18ceba51e3bc5c6c3b3aaf98f5e0e8dadd2969d450af4e.jpg...
Copying gs://tfmbucket1/images/train/LowRes/LowRes_5f307b54d3eb1f855dd8165d11ed0735b03c06526a5053e29c143e4128fe6f68.jpg...
Copying gs://tfmbucket1/images/train/LowRes/LowRes_5f307c7812fa0f230bfa2bd7c76462d5539b07a0099bf65c846449160b8ce8be.jpg...
Copying gs://tfmbucket1/images/train/LowRes/LowRes_5f308af0f826c664b2529ec3734ac458642e6235612ff55a2ebd6ca9d13867cd.jpg...
Copying gs://tfmbucket1/images/train/LowRes/LowRes_5f309b2491b92290ccf27192ff5dbd5c277b2e8fbe9323216afe6a92f88a61f7.jpg...
Copying gs://tfmbucket1/images/train/LowRes/LowRes_5f309fde4dc9ef84e17d0f9e2cc

In [ ]:
#Tamaño de las imagenes LowRes en Drive del proyecto
#! du -sh /content/drive/My\ Drive'/TFM Big Data & BA/Propuestas/Proyecto Final/4 - Deep Learning/LowRes'
! du -sh '/content/drive/My Drive/TFM Big Data & BA/Propuestas/Proyecto Final/4 - Deep Learning/LowRes'

813M	/content/drive/My Drive/TFM Big Data & BA/Propuestas/Proyecto Final/4 - Deep Learning/LowRes


In [ ]:
# Una vez subidas las fotos, contamos el numero total de fotos encontradas
! ls -lR '/content/drive/My Drive/TFM Big Data & BA/Propuestas/Proyecto Final/4 - Deep Learning/LowRes' | tail -n 1 

-rw------- 1 root root 3614 Jan 14 22:01 LowRes_3b07f24847dcb19cfce70fe141ab726f03a1b944a6566815b54ae54645cf88c3.jpg


In [ ]:
#Tamaño de las imagenes LowRes en Cloud Storage
!gsutil du -sh gs://tfmbucket1/images/train/LowRes

3.24 GiB     gs://tfmbucket1/images/train/LowRes


In [ ]:
# Una vez subidas las fotos, contamos el numero total de fotos encontradas
!gsutil ls -lR gs://tfmbucket1/images/train/LowRes_V2 | tail -n 1 

TOTAL: 612647 objects, 2422422583 bytes (2.26 GiB)


In [ ]:
# Una vez subidas las fotos, contamos el numero total de fotos encontradas
!gsutil ls -lR gs://tfmbucket1/images/train/LowRes | tail -n 1 

In [ ]:
#Tamaño de las imagenes cleaned
# !gsutil du -sh gs://tfmbucket17/images/test/cleaned

36.37 GiB    gs://tfmbucket17/images/test/cleaned


### Vamos a filtrar registros con el dataset nuevo train_data.csv

# Downsizing

In [ ]:
# Contamos el número de imagenes seleccionadas en funcion de las categorias
# len(df['image'])

In [ ]:
# Una vez subidas las fotos, contamos el numero total de fotos encontradas
!gsutil ls -lR gs://tfmbucket17/images/test/cleaned | tail -n 1 

TOTAL: 1039013 objects, 39055135397 bytes (36.37 GiB)


En total hay 1064954 (df) - 1039013 (cleaned) = 25.941 anuncios que hemos decidido mantener pero para los que no se ha encontrado imagen. Compruebo si existe correlación con alguna categoría en particular o puede ser debido a algún filtro de año o mierdas similares que hayan hecho los rusos.

In [ ]:
# Me bajo un index de los archivos
# !gsutil ls gs://tfmbucket17/images/test/cleaned* > /content/cleaned_images_index.txt

In [ ]:
# Lo subo a GCP para recuperarlo rapido luego
# !gsutil cp /content/cleaned_images_index.txt gs://tfmbucket17/files/cleaned_images_index.txt

Copying file:///content/cleaned_images_index.txt [Content-Type=text/plain]...
\
Operation completed over 1 objects/105.0 MiB.                                    


In [ ]:
!gsutil cp gs://tfmbucket17/files/cleaned_images_index.txt /content/cleaned_images_index.txt 

data = pd.read_csv('/content/cleaned_images_index.txt', sep=" ", header= None)
data.columns = ["path"]
data['image']  = data.apply(lambda x: x['path'].split('/')[6], axis = 1)
data['LowRes'] = data.apply(lambda x: '/content/converted_files/LowRes/LowRes_' + x['path'].split('/')[6], axis = 1)

Copying gs://tfmbucket17/files/cleaned_images_index.txt...
\ [1 files][105.0 MiB/105.0 MiB]                                                
Operation completed over 1 objects/105.0 MiB.                                    


In [ ]:
# Subset para evitar overflow de colab
nchunks = 10000

chunk = math.ceil(1039013/nchunks)
dfs = {}
for n in range((data.shape[0] // chunk + 1)):
    df_temp = data.iloc[n*chunk:(n+1)*chunk]
    df_temp = df_temp.reset_index(drop=True)
    dfs[n] = df_temp

if data.shape[0] % chunk != 0:
    df_temp = data.iloc[-int(data.shape[0] % chunk):]
    df_temp = df_temp.reset_index(drop=True)
    dfs[n] = df_temp
else:
    pass

In [ ]:
!mkdir /content/indexes
!mkdir /content/extracted_files
!mkdir /content/converted_files
!mkdir /content/converted_files/LowRes
#!mkdir /content/converted_files/BWLowRes

In [ ]:
# Ciclo 1: 0-150      OK 9/01
# Ciclo 2: 151-1001   OK 10/01
# Ciclo 2: 1002-3001  OK 11/01
# Ciclo 3: 3002-3511  OK 10/01
# Ciclo 4: 3502-3773  OK 10/01
# Ciclo 5: 3774-4000  OK 11/01  
# Ciclo 6: 4001-4643  OK 11/01
# Ciclo 7: 6501-6666  OK 11/01
# Ciclo 7: 7200-7300  OK 11/01
# Ciclo 8: 7000-7139  OK 11/01
# Ciclo 9: 

for i in range(7239, 7301):
  start = timeit.default_timer()
  np.savetxt(r'/content/indexes/cleaned_images_index_' + str(i) + '.txt', dfs[i]['path'], fmt='%s')
  np.savetxt(r'/content/indexes/LowRes_cleaned_images_index_' + str(i) + '.txt', dfs[i]['LowRes'], fmt='%s')

  actual_path = '/content/indexes/cleaned_images_index_' + str(i) + '.txt'
  final_path  = '/content/indexes/LowRes_cleaned_images_index_' + str(i) + '.txt'

  !cat {actual_path} | gsutil -mq cp -I /content/extracted_files/.
  #print('Files copied for cicle ' + str(i))

  for img in dfs[i]['image']:
    imgfile = Image.open('/content/extracted_files/' + img) # image extension *.png,*.jpg
    new_width  = 128
    new_height = 128
    imgfile = imgfile.resize((new_width, new_height), Image.ANTIALIAS)
    imgfile.save('/content/converted_files/LowRes/LowRes_' + img)

    #imgGray = imgfile.convert('L')
    #imgGray.save('/content/converted_files/BWLowRes/BWLowRes_' + img)
    
  !cat {final_path} | !gsutil -mq cp -r gs://tfmbucket17/images/test/LowRes /content/imagenes_deepl/.
  stop = timeit.default_timer()
  print('Files compressed and uploaded for cicle ', str(i), 'in ', str(round(stop - start,2)), 's | ETA: ', str(round((stop-start)*(7301-i)/60,2)), 'min')

# Limpio directorios
  !rm -r /content/converted_files/LowRes
  !rm -r /content/extracted_files
# Creo para el siguiente ciclo
  !mkdir /content/converted_files/LowRes
  !mkdir /content/extracted_files

Files compressed and uploaded for cicle  7239 in  16.81 s | ETA:  17.37 min
Files compressed and uploaded for cicle  7240 in  16.49 s | ETA:  16.76 min
Files compressed and uploaded for cicle  7241 in  16.63 s | ETA:  16.63 min
Files compressed and uploaded for cicle  7242 in  16.62 s | ETA:  16.34 min
Files compressed and uploaded for cicle  7243 in  17.01 s | ETA:  16.44 min
Files compressed and uploaded for cicle  7244 in  16.82 s | ETA:  15.98 min
Files compressed and uploaded for cicle  7245 in  16.8 s | ETA:  15.68 min
Files compressed and uploaded for cicle  7246 in  17.25 s | ETA:  15.81 min
Files compressed and uploaded for cicle  7247 in  17.33 s | ETA:  15.59 min
Files compressed and uploaded for cicle  7248 in  17.09 s | ETA:  15.1 min
Files compressed and uploaded for cicle  7249 in  16.9 s | ETA:  14.64 min
Files compressed and uploaded for cicle  7250 in  17.02 s | ETA:  14.47 min
Files compressed and uploaded for cicle  7251 in  17.21 s | ETA:  14.34 min
Files compresse

In [ ]:
import os
import tensorflow as tf
from tensorflow.python.platform import gfile
filelist = gfile.ListDirectory("gs://tfmbucket17/images/test/cleaned/")
sess = tf.compat.v1.Session()
with sess.as_default():
   x = np.array([np.array(tf.image.decode_png(tf.compat.v1.read_file(os.path.join(train_files_dir, filename))).eval()) for filename in filelist])

In [ ]:
# Load image
image_path = 'gs://xxxxxxx.jpg'
image = tf.read_file(image_path)
image = tf.image.decode_jpeg(image)
image_array = sess.run(image)

# Save iamge
job_dir = 'gs://xxxxxxxx'
image = tf.image.encode_jpeg(image_array)
file_name = 'xxx.jpg'
write_op = tf.write_file(os.path.join(job_dir, file_name), image)
sess.run(write_op)

In [ ]:
# Preparar imagenes para subir a Vertex
https://cloud.google.com/vertex-ai/docs/datasets/prepare-image

# Una vez preparados, generar un archivo de importación para crear el conjuto de datos
https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_image_sample-python


# VGGNet
Red para reconocimiento de objetos (preentrenada para mas de 1000 objetos)

In [ ]:
https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/

# Spare code

In [ ]:
# ---------------------- SPARE CODE -------------------------

# posibilidad de crear un zip con las imagenes reducidas
import shutil
shutil.make_archive('gs://tfmbucket17/images/test/train_jpg_filtered', 'zip', '/content/extractions/')

# Para limpiar directorios locales
import os
dir_name = "/content/"
test = os.listdir(dir_name)

for item in test:
    if item.endswith(".jpg"):
        os.remove(os.path.join(dir_name, item))

In [ ]:
import os
import numpy as np
import cv2
obj=[]
img_size=150
obj_folder_path="gs://tfmbucket17/images/test"

for img in os.listdir(obj_folder_path):
    img = cv2.imread(os.path.join(obj_folder_path,img))
    img_resize= cv2.resize(img,(img_size,img_size))
    obj.append(img_resize)